In [ ]:
##### Average Kinetic Energy and Temperature
# Written by Joseph J. Williams under the guidance of James H. Cooley and Michael Murillo 2017/06/02

# VARIABLES
# m = mass of a particle
# k = Boltzmann's Constant
# M = total number of time steps
# N = total number of particles
# momentum = a 2-array whose (i,j) point is the momentum of particle j at time i,
    # i.e. row x of 'momentum' contains the momentum of every particle at time x
    # i.e. momentum.shape should return (M,N)
# E_k = average over time of the kinetic energy of the system
# T = temperature

# E_k = 1/(2m) * (sum of each particle's momentum at every point in time) / total time steps
# T = E_k * 2/3 / N / k
# See page 226 of Molecular Dynamics Simulation by Haile.

# Also assuming that the mass of every particle is the same.

# Using metric units.

import numpy as np
k = 1.38064852 * 10^(-23) # m^2*kg / (s^2 *K), meters squared kilogram, per second squared Kelvin

E_k = 0.5/m/M * sum(sum(momtentum*momentum))
T = E_k * 2/3 / N / k

In [ ]:
##### Configurational Energy
# Written by Joseph J. Williams under the guidance of James H. Cooley and Michael Murillo 2017/06/02

# VARIABLES
# M = total number of time steps
# N = total number of particles
# potential = a 3-array whose (i,j,k) point is the intermolecular potential energy between
    # pair j of 2 particular particles at time i, considering cell translation vector k.
    # potential.shape should return (M,N_choose_2,D), where D is the square of the dimension of the space 
    # in which we're simulating (in our case, D = 1), and N_choose_2 = scipy.special.comb(N,2).
    # There are N choose 2 unique pairs of particles, so there are as many potential energies.
# U = average over time of the configurational energy of the system

# U = (sum over time of the sum over each pair of particle's potential energy, considering 
# all particles and their images) / total time steps
# See page 226 of Molecular Dynamics Simulation by Haile.

# Note that the formula presented there may appear rather menacing at first blush due to the four summations.
# The first summation is a summation over time. The third and fourth summations go together, and are shorthand
# for instructing the reader to not repeat any pairs of particles, i.e. to NOT consider the potential energy
# between particles e.g. i=5 and j=1, because that potential energy was already calculated when i=1 and j=5;
# further, because it is written i<j, as opposed to i<=j, we are also not computing the potential energy between 
# a particle and itself, which is as it should be.

# The second summation, a summation over alpha, is related to the periodic boundary conditions; it is 
# instructing the reader, when considering particle i, to consider its potential energy with both particle j
# and all images of j. The value of alpha is different for each image of j. In our 1-D case, there will be just
# two images of each atom, so there will be three values of alpha: -1, 0, and 1. L is the length of the domain.
# Finally, note that we (of course) do not consider the potential between a particle and itself, but less 
# obviously we do not consider the potential between a particle and any of its own images.
# See pages 81-86 of Molecular Dynamics Simulation by Haile for discussion regarding periodic boundary conditions.

# While the aforementioned pages clearly state that it should be so, I admit that it seems strange to me to 
# consider the potential both between particles i and j and between particle i and all images of j; it suggests
# an understanding of periodic boundary conditions (PBCs) different from my own. Previously, I had understood PBCs
# to glue (in the topological sense) the left and right edges and the top and bottom edges. Haile, however,
# seems to suggest that we should consider image domains beyond the main domain, and consider the existence
# of not N particles, but (in our 1-D case) 3N particles, 2N of which are images.

U = 1/M * sum(potential)

In [ ]:
##### Potential Energies
# Written by Joseph J. Williams under the guidance of James H. Cooley and Michael Murillo 2017/06/02

# VARIABLES
# L = length of the domain
# sigma = the finite distance at which the inter-particle potential is zero
# eps = the depth of the potential well, or the energy at the minimum in u(r)
# position = a 2-array whose (i,j) point is the position of particle j at time i,
    # i.e. row x of 'position' contains the position of every particle at time x
    # i.e. position.shape should return (M,N)
# distance = a 2-array whose entries are the distances between particular pairs of particles
# potential = a 3-array whose (i,j,k) point is the intermolecular potential energy between
    # pair j of 2 particular particles at time i, considering cell translation vector k.
    # potential.shape should return (M,N_choose_2,D), where D is the square of the dimension of the space 
    # in which we're simulating (in our case, D = 1), and N_choose_2 = scipy.special.comb(N,2).
    # There are N choose 2 unique pairs of particles, so there are as many potential energies.
# alphas = a 1-array (a vector) whose entries are -1, 0, and 0. These are the cell translation vectors for a 1-D case.


# Precise details of this code depend upon the intermolecular potential energy function.
# For now, I am assuming the Lennard-Jones model presented on pages 189 and 190 of of Molecular Dynamics Simulation by Haile:
# u(r) = 4*eps*((sigma/r)^12 - (sigma/r)^6)


alphas = [-1, 0, 1]

#We have three values in alpha, so we must calculate potentials three times
distance = np.zeros( (int(scipy.special.comb(N,2)),1) )
u_1 = np.zeros( (int(scipy.special.comb(N,2)),1,1) )
u_2 = np.zeros( (int(scipy.special.comb(N,2)),1,1) )
u_3 = np.zeros( (int(scipy.special.comb(N,2)),1,1) )

k=0
for i in range(0,N):
    for j in range(i,N):
        if i != j:
            distance[k] = abs(position[j] - position[i]) - L*alphas[0]
            u_1[k] = 4*eps*((sigma/distance[k][0])**12 - (sigma/distance[k][0])**6)
            k=k+1
            
k=0
for i in range(0,N):
    for j in range(i,N):
        if i != j:
            distance[k] = abs(position[j] - position[i]) - L*alphas[1]
            u_2[k] = 4*eps*((sigma/distance[k][0])**12 - (sigma/distance[k][0])**6)
            k=k+1
            
k=0
for i in range(0,N):
    for j in range(i,N):
        if i != j:
            distance[k] = abs(position[j] - position[i]) - L*alphas[2]
            u_3[k] = 4*eps*((sigma/distance[k][0])**12 - (sigma/distance[k][0])**6)
            k=k+1
            
u = u_1 + u_2 + u_3

In [ ]:
##### Total Energy
# Written by Joseph J. Williams under the guidance of James H. Cooley and Michael Murillo 2017/06/02

# VARIABLES
# E_k = average over time of the kinetic energy of the system
# U = average over time of the configurational energy of the system
# E = total energy
    # Note that total energy should be synonymous with average total energy over time, hence why
    # it is appropriate to add the average kinetic and configurational energies together to find
    # total energy. We are considering an isolated system, and the total energy should not vary 
    # over time, except, perhaps, by nearly infinitesimal quantities due to floating-point errors.

# See pages 225-226 of Molecular Dynamics Simulation by Haile.

E = E_k + U